In [31]:
import pandas as pd 
import numpy as np
import glob
import cv2 as cv
import os
import json
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt

from pathlib import Path
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split


import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.models import resnet18
from PIL import Image
from IPython.display import clear_output


from core.model import BaselineModel, BaselineWithOriginalAttentionModel
from core.dataset import PredictImageDataset
from core.utils import sc_to_angle



from run_train import orig_transform, valid_transform

In [32]:
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Resize(224),
#     transforms.Normalize(
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225]),
# ])


def read_img(p, scale=4, tf=valid_transform):
    im = Image.open(p)
    linear, _ = im.size
    im = np.array(im.resize((linear // scale, linear // scale)))
    im = tf(im).unsqueeze(0).to(device).float()
    return im


def process_output(out):
    x, y, s, c = out.detach().cpu().numpy()
    x = int(round((x + 1) / 2. * 10496))
    y = int(round((y + 1) / 2. * 10496))
    return x, y, s.item(), c.item()

In [33]:
device = 'cuda:0'

In [34]:
# BaselineWithOriginalAttentionModel(out_ch=4, attn_dim=512)
# 'best_vanilla_crossattn_500_epoch.pt'

In [35]:
model_path = 'models/best_model.pt'

ckpt = torch.load(model_path)
model = BaselineModel(out_ch=5, pretrained=False)
model.load_state_dict(ckpt, strict=True)
model.to(device)
model.eval()

print()

In [36]:
def center_angle_to_default(c, deg_angle):
    cc = np.array(c)
    
    rad_angle = np.deg2rad(deg_angle)
    M = np.array([
        [np.cos(rad_angle), -np.sin(rad_angle)],
        [np.sin(rad_angle), np.cos(rad_angle)]
    ])

    
    lb = cc + M @ np.array([-512, 512]) # left bottom
    lt = cc + M @ np.array([-512, -512]) # left top
    rt = cc + M @ np.array([512, -512]) # right top
    rb = cc + M @ np.array([512, 512]) # right bottom
    
    
    return {
        'left_top': np.round(lt).astype(np.int32).tolist(), 
        'right_top': np.round(rt).astype(np.int32).tolist(),
        'left_bottom': np.round(lb).astype(np.int32).tolist(),
        'right_bottom': np.round(rb).astype(np.int32).tolist(),
        'angle': deg_angle
    }


def default_to_center_angle(obj):
    center = (
        np.array(obj['left_top']) + \
        np.array(obj['right_bottom']) + \
        np.array(obj['left_bottom']) + \
        np.array(obj['right_top'])
    ) / 4
    
    return np.array(center.tolist() + [np.deg2rad(obj['angle'])])


def read_img_data(path):
    with open(path, 'r') as f:
        obj = json.load(f)
    
    
    return obj, default_to_center_angle(obj)

In [37]:
path = 'train/json/1.json'

obj, (x, y, angle) = read_img_data(path)

obj, x, y, angle

({'left_top': [8533, 2184],
  'right_top': [9501, 1851],
  'left_bottom': [8866, 3152],
  'right_bottom': [9834, 2819],
  'angle': 341},
 9183.5,
 2501.5,
 5.951572749300664)

In [38]:
for k, value in obj.items():
    if k == 'angle':
        continue
    
    initial = np.array(value) - np.array([x, y])
    
    M = np.array([
        [np.cos(angle), np.sin(angle)],
        [-np.sin(angle), np.cos(angle)]
    ])
    
    print(k, M @ initial)

left_top [-511.69194439 -511.98423223]
right_top [ 511.98423223 -511.69194439]
left_bottom [-511.98423223  511.69194439]
right_bottom [511.69194439 511.98423223]


In [39]:
restored_obj = center_angle_to_default([x, y], np.rad2deg(angle))

obj, restored_obj

({'left_top': [8533, 2184],
  'right_top': [9501, 1851],
  'left_bottom': [8866, 3152],
  'right_bottom': [9834, 2819],
  'angle': 341},
 {'left_top': [8533, 2184],
  'right_top': [9501, 1851],
  'left_bottom': [8866, 3152],
  'right_bottom': [9834, 2819],
  'angle': 341.0})

In [40]:
x, y, rad = default_to_center_angle(restored_obj)
x, y, np.rad2deg(rad)

(9183.5, 2501.5, 341.0)

In [41]:
# p = 'test_images/3.png'
# im = read_img(p)
orig_im = read_img('original.tiff', tf=orig_transform)

In [42]:
dataset = PredictImageDataset('test_images', transform=valid_transform)
pred_loader = torch.utils.data.DataLoader(dataset, batch_size=8, num_workers=8, pin_memory=True)

In [43]:
results = {}
out_type = 'xyxya'

for names, imgs in tqdm(pred_loader):
    imgs = imgs.to(device).float()
    out = model(orig_im, imgs)
    out = out.cpu().detach().numpy()
    
    if out_type == 'xyxya':
        out = np.clip(out, 0, 1)
        
        ltxy = out[:, :2] * 10496
        rbxy = out[:, 2:4] * 10496
        
        xy = (ltxy + rbxy) / 2
        angle = np.rad2deg(out[:, 4] * 2 * np.pi)
        
    else:
        xy = np.round((out[:, :2] + 1) / 2 * 10496).astype(np.int32)
        angle = sc_to_angle(out[:, 2], out[:, 3])
        # angle = np.rad2deg(((out[:, 2]) + 1) * np.pi).astype(np.int32)
    pred = np.hstack([xy, np.expand_dims(angle, 1)])
    
    for idx, name in enumerate(names):
        results[name] = center_angle_to_default(pred[idx, :2].tolist(), pred[idx, 2])

  0%|          | 0/50 [00:00<?, ?it/s]

In [10]:
def read_submit(root):
    res = {}
    
    for p in Path(root).iterdir():
        with p.open('r') as f:
            obj = json.load(f)
        
        res[p.stem] = obj
    return res

In [44]:
!rm submit/*

In [45]:
fin = {}

for name, obj in results.items():
    obj['angle'] = int(round(obj['angle']))
    fin[name] = obj

In [46]:
for k, data in fin.items():
    name = f'submit/{k}.json'
    
    
    with open(name, 'w') as f:
        f.write(json.dumps(data))

In [19]:
!zip -r sub.zip submit/*.json

/bin/bash: zip: command not found
